In [92]:
import pandas as pd
import os
import shutil
from pathlib import Path
import logging
from time import time

In [3]:
folder_ter = Path(r"C:\Users\ldecarva\Documents\Data\Danube\Carte_territoire")

In [9]:
path_ter = Path(*folder_ter.parts[:-2]) / "Danube" / "Carte_territoire" / "territ_dept.xlsx"
path_ter

WindowsPath('C:/Users/ldecarva/Documents/Data/Danube/Carte_territoire/territ_dept.xlsx')

In [11]:
df = pd.read_excel(path_ter)
df.head()

,INSEE_DEP,Part_Tui,Part_Ard,Part_Zin,Dens_toit,Toit,P1Old,P2Old,Terr_P1,Terr_P2,Diff_P1
0,01,0.935484,0.024831,0.039684,0.006175,Tuile,FRANCE_TERRE_TUILE,FRANCE_TUILE,FRANCE_TERRE_TUILE,FRANCE_TUILE,1
1,09,0.780596,0.174623,0.029491,0.002621,Tuile,FRANCE_PIERRE_SCHISTE_TUILE,FRANCE_TUILE,FRANCE_PIERRE_SCHISTE_TUILE,FRANCE_TUILE,1
2,11,0.959668,0.033371,0.006960,0.002958,Tuile,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE,1
3,12,0.441912,0.533258,0.024830,0.002021,Ardoise,FRANCE_PIERRE_SCHISTE_ARDOISE,FRANCE_ARDOISE,FRANCE_PIERRE_GRANITE_ARDOISE,FRANCE_ARDOISE,1
4,13,0.994355,0.001509,0.004136,0.011913,Tuile,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE,1


In [12]:
for col in df.columns:
    print("\n",col)
    uniq = df[col].unique()
    print(len(uniq))
    print(uniq)


 INSEE_DEP
96
['01' '09' '11' '12' '13' '14' '15' '16' '17' '18' '19' '10' '2A' '2B'
 '21' '22' '23' '24' '25' '26' '27' '28' '02' '29' '30' '31' '32' '33'
 '34' '35' '36' '37' '38' '03' '39' '40' '41' '42' '43' '44' '45' '46'
 '47' '48' '04' '49' '50' '51' '52' '53' '54' '55' '56' '57' '05' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '06' '68' '69' '70' '71'
 '72' '73' '74' '75' '76' '77' '07' '78' '79' '80' '81' '82' '83' '84'
 '85' '86' '87' '08' '88' '89' '90' '91' '92' '93' '94' '95']

 Part_Tui
96
[0.93548437 0.78059606 0.95966803 0.44191235 0.99435524 0.4980702
 0.56026578 0.97343438 0.97522845 0.74837633 0.37467296 0.95146519
 0.99483574 0.65945269 0.94379305 0.01558085 0.66277667 0.94267935
 0.95593592 0.56694291 0.65089615 0.76047358 0.53830666 0.01521947
 0.97737879 0.89296515 0.97916256 0.97004724 0.97254006 0.01247856
 0.72789999 0.46346855 0.86804242 0.85268932 0.95683363 0.9658959
 0.70269115 0.96193851 0.93423138 0.40016614 0.75318205 0.92353417
 0.98191101 0.42

In [13]:
equal_P1 = set(df[df.Terr_P1 == df.P1Old].INSEE_DEP)
diff_P1 = set(df[df.Terr_P1 != df.P1Old].INSEE_DEP)
equal_P2 = set(df[df.Terr_P2 == df.P2Old].INSEE_DEP)
diff_P2 = set(df[df.Terr_P2 != df.P2Old].INSEE_DEP)

In [14]:
diff_P1 - diff_P2

{'12', '37', '48'}

In [15]:
len(diff_P1)

27

In [16]:
diff_table_P1 = set(df[df["Diff_P1"]==0]["INSEE_DEP"].unique())
len(diff_table_P1)

24

In [17]:
diff_P1 - diff_table_P1

{'12', '37', '48'}

In [18]:
# The difference between the two sets is because of the propositions for solving the territory problem
df[df["INSEE_DEP"]=="12"]

,INSEE_DEP,Part_Tui,Part_Ard,Part_Zin,Dens_toit,Toit,P1Old,P2Old,Terr_P1,Terr_P2,Diff_P1
3,12,0.441912,0.533258,0.02483,0.002021,Ardoise,FRANCE_PIERRE_SCHISTE_ARDOISE,FRANCE_ARDOISE,FRANCE_PIERRE_GRANITE_ARDOISE,FRANCE_ARDOISE,1


In [19]:
diff_P2 - diff_P1

{'75'}

In [20]:
diff_P1.symmetric_difference(diff_P2)

{'12', '37', '48', '75'}

In [21]:
diff = diff_P2
diff.update(diff_P1)

In [22]:
diff_P1 - diff

set()

In [23]:
diff_P2 - diff

set()

In [24]:
len(diff)

28

In [25]:
len(diff)/len(df) 

0.2916666666666667

In [26]:
diff

{'02',
 '03',
 '08',
 '12',
 '14',
 '18',
 '23',
 '27',
 '28',
 '37',
 '41',
 '45',
 '48',
 '53',
 '58',
 '60',
 '61',
 '62',
 '64',
 '65',
 '67',
 '72',
 '73',
 '75',
 '78',
 '80',
 '86',
 '87'}

In [27]:
df_diff = df[df["INSEE_DEP"].isin(diff)][["INSEE_DEP","Terr_P1","Terr_P2"]].set_index("INSEE_DEP")
df_diff

,Terr_P1,Terr_P2
INSEE_DEP,,
12,FRANCE_PIERRE_GRANITE_ARDOISE,FRANCE_ARDOISE
14,FRANCE_PIERRE_GRANITE_TUILE,FRANCE_TUILE
18,FRANCE_PIERRE_GRES_TUILE,FRANCE_TUILE
23,FRANCE_PIERRE_GRANITE_TUILE,FRANCE_TUILE
27,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE
28,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE
02,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE
37,FRANCE_PIERRE_CALCAIRE_ARDOISE,FRANCE_ARDOISE
03,FRANCE_PIERRE_GRES_TUILE,FRANCE_TUILE


In [28]:
dict_territ = df_diff.to_dict()
dict_territ

{'Terr_P1': {'12': 'FRANCE_PIERRE_GRANITE_ARDOISE',
  '14': 'FRANCE_PIERRE_GRANITE_TUILE',
  '18': 'FRANCE_PIERRE_GRES_TUILE',
  '23': 'FRANCE_PIERRE_GRANITE_TUILE',
  '27': 'FRANCE_PIERRE_CALCAIRE_TUILE',
  '28': 'FRANCE_PIERRE_CALCAIRE_TUILE',
  '02': 'FRANCE_PIERRE_CALCAIRE_TUILE',
  '37': 'FRANCE_PIERRE_CALCAIRE_ARDOISE',
  '03': 'FRANCE_PIERRE_GRES_TUILE',
  '41': 'FRANCE_PIERRE_CALCAIRE_TUILE',
  '45': 'FRANCE_BRIQUE_TUILE',
  '48': 'FRANCE_PIERRE_CALCAIRE_ARDOISE',
  '53': 'FRANCE_BRIQUE_ARDOISE',
  '58': 'FRANCE_TERRE_TUILE',
  '60': 'FRANCE_PIERRE_CALCAIRE_TUILE',
  '61': 'FRANCE_BRIQUE_TUILE',
  '62': 'FRANCE_BRIQUE_TUILE',
  '64': 'FRANCE_PIERRE_GRES_TUILE',
  '65': 'FRANCE_BRIQUE_TUILE',
  '67': 'FRANCE_PIERRE_GRES_TUILE',
  '72': 'FRANCE_BRIQUE_TUILE',
  '73': 'FRANCE_BOIS_TUILE',
  '75': 'FRANCE_PIERRE_CALCAIRE_ZINC',
  '78': 'FRANCE_BRIQUE_TUILE',
  '80': 'FRANCE_BRIQUE_TUILE',
  '86': 'FRANCE_PIERRE_CALCAIRE_TUILE',
  '87': 'FRANCE_PIERRE_GRANITE_TUILE',
  '08': 'FRANCE

### Correcting territories

In [36]:
folder_save = r"C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept"

In [19]:
folder_csvs = r"C:\Users\ldecarva\Documents\Data\gpd_sorting_output\archetypes_per_dept_tables"
list_csvs = listdir(folder_csvs)

In [44]:
for csv_name in list_csvs:
    # Source path
    source = fr"{folder_csvs}\{csv_name}"
    # Destination path
    destination = fr"{folder_save}\{csv_name}"
    dept = csv_name.split("-")[4][3:]
    
    if dept in diff:
        print(dept, " : CHANGED")
        file = pd.read_csv(source)
        file["Ter_P1_min"] = dict_territ["Terr_P1"][dept]
        file["Ter_P2-7_min"] = dict_territ["Terr_P2"][dept]
        file.to_csv(destination)
        
    else:
        try:
        # Copying the other files without alteration
            shutil.copy(source, destination)
            print(dept, "File copied successfully.")
        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represents the same file.")
        # If there is any permission issue
        except PermissionError:
            print("Permission denied.")
        # For other errors
        except:
            print("Error occurred while copying file.")

31 File copied successfully.
59 File copied successfully.
34 File copied successfully.
42 File copied successfully.
64  : CHANGED
81 File copied successfully.
74 File copied successfully.
59 File copied successfully.
60  : CHANGED
69 File copied successfully.
78  : CHANGED
59 File copied successfully.
19 File copied successfully.
06 File copied successfully.
69 File copied successfully.
73  : CHANGED
40 File copied successfully.
74 File copied successfully.
37  : CHANGED
45  : CHANGED
61  : CHANGED
62  : CHANGED
65  : CHANGED
72  : CHANGED
76 File copied successfully.
78  : CHANGED
80  : CHANGED
21 File copied successfully.
31 File copied successfully.
53  : CHANGED
54 File copied successfully.
59 File copied successfully.
71 File copied successfully.
79 File copied successfully.
85 File copied successfully.
91 File copied successfully.
92 File copied successfully.
93 File copied successfully.
94 File copied successfully.
95 File copied successfully.
02  : CHANGED
19 File copied succes

46 File copied successfully.
55 File copied successfully.
69 File copied successfully.
70 File copied successfully.
77 File copied successfully.
83 File copied successfully.
75  : CHANGED
47 File copied successfully.
82 File copied successfully.
84 File copied successfully.
14  : CHANGED
23  : CHANGED
29 File copied successfully.
49 File copied successfully.
56 File copied successfully.
87  : CHANGED
2A File copied successfully.
2B File copied successfully.
03  : CHANGED
18  : CHANGED
22 File copied successfully.
35 File copied successfully.
44 File copied successfully.
50 File copied successfully.
64  : CHANGED
67  : CHANGED
07 File copied successfully.
43 File copied successfully.
57 File copied successfully.
81 File copied successfully.
88 File copied successfully.
08  : CHANGED
05 File copied successfully.
10 File copied successfully.
24 File copied successfully.
26 File copied successfully.
68 File copied successfully.
12  : CHANGED
48  : CHANGED
09 File copied successfully.
32 Fi

C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (190) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


40 File copied successfully.
74 File copied successfully.
37  : CHANGED
45  : CHANGED


C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (146,290,292) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (28,44,45,66,124,146,239,294) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


54 File copied successfully.
55 File copied successfully.
57 File copied successfully.
59 File copied successfully.
63 File copied successfully.
66 File copied successfully.
68 File copied successfully.
69 File copied successfully.
70 File copied successfully.
71 File copied successfully.
74 File copied successfully.
75  : CHANGED
77 File copied successfully.
79 File copied successfully.
81 File copied successfully.
82 File copied successfully.
83 File copied successfully.
84 File copied successfully.
85 File copied successfully.
88 File copied successfully.
89 File copied successfully.
90 File copied successfully.
91 File copied successfully.
92 File copied successfully.
93 File copied successfully.
94 File copied successfully.
95 File copied successfully.
22 File copied successfully.
27  : CHANGED
28  : CHANGED
37  : CHANGED
44 File copied successfully.
45  : CHANGED
49 File copied successfully.
50 File copied successfully.
56 File copied successfully.
62  : CHANGED
64  : CHANGED
78 

83 File copied successfully.
84 File copied successfully.
90 File copied successfully.
91 File copied successfully.
92 File copied successfully.
93 File copied successfully.
94 File copied successfully.
95 File copied successfully.
14  : CHANGED
37  : CHANGED
44 File copied successfully.
45  : CHANGED
49 File copied successfully.
56 File copied successfully.
60  : CHANGED
62  : CHANGED
64  : CHANGED
67  : CHANGED
72  : CHANGED
73  : CHANGED
76 File copied successfully.
78  : CHANGED
80  : CHANGED
86  : CHANGED
01 File copied successfully.
06 File copied successfully.
07 File copied successfully.
10 File copied successfully.
13 File copied successfully.
16 File copied successfully.
17 File copied successfully.
21 File copied successfully.
25 File copied successfully.
30 File copied successfully.
31 File copied successfully.
33 File copied successfully.
34 File copied successfully.
38 File copied successfully.
42 File copied successfully.
51 File copied successfully.
57 File copied succe

79 File copied successfully.
81 File copied successfully.
82 File copied successfully.
83 File copied successfully.
84 File copied successfully.
85 File copied successfully.
88 File copied successfully.
89 File copied successfully.
90 File copied successfully.
91 File copied successfully.
92 File copied successfully.
93 File copied successfully.
94 File copied successfully.
95 File copied successfully.
02  : CHANGED
03  : CHANGED
08  : CHANGED
12  : CHANGED
14  : CHANGED
15 File copied successfully.
18  : CHANGED
19 File copied successfully.
22 File copied successfully.
23  : CHANGED
27  : CHANGED
28  : CHANGED
29 File copied successfully.
30 File copied successfully.
35 File copied successfully.
37  : CHANGED
41  : CHANGED
44 File copied successfully.
45  : CHANGED


KeyboardInterrupt: 

### Check territories 

In [68]:
# checking territory
folder_new_csvs = r"C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept"
list_csvs = os.listdir(folder_new_csvs)
problem_ter = []

for csv_name in list_csvs:
    file = pd.read_csv(fr"{folder_new_csvs}\{csv_name}",nrows=1)
    (file.columns)
    dept_nb = str(file.territory.unique()[0])

    dept_name = csv_name.split("-")[3]

    print(dept_nb, dept_name)
    
    if dept_nb != dept_name:
        
        print(csv_name)
        display(file[["INSEE_DEP_min", 'typology',
             'usage',
             'territory',
             'period','Ter_P1_min',
             'Ter_P2-7_min',]].head())
        problem_ter.append(csv_name)
        

FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_BOIS_TUILE FRANCE_BOIS_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_PIERRE_CALCAIRE_ARDOISE FRANCE_PIERRE_CALCAIRE_ARDOISE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_BOIS_TUILE FRANCE_BOIS_TUILE
FRANCE_BOIS_TUILE FRANCE_BOIS_TUILE
FRANCE_BOIS_TUILE FRANCE_BOIS_TUILE
FRANCE_BRIQUE_ARDOISE FRANCE_BRIQUE_ARDOISE
FRANCE_BRIQUE_ARDOISE FRANCE_BRIQUE_ARDOISE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUIL

FRANCE_PIERRE_GRANITE_ARDOISE FRANCE_PIERRE_GRANITE_ARDOISE
FRANCE_PIERRE_GRANITE_ARDOISE FRANCE_PIERRE_GRANITE_ARDOISE
FRANCE_PIERRE_GRANITE_TUILE FRANCE_PIERRE_GRANITE_TUILE
FRANCE_PIERRE_GRANITE_TUILE FRANCE_PIERRE_GRANITE_TUILE
FRANCE_PIERRE_GRANITE_TUILE FRANCE_PIERRE_GRANITE_TUILE
FRANCE_PIERRE_GRANITE_TUILE FRANCE_PIERRE_GRANITE_TUILE
FRANCE_PIERRE_GRANITE_TUILE FRANCE_PIERRE_GRANITE_TUILE
FRANCE_PIERRE_GRES_ARDOISE FRANCE_PIERRE_GRES_ARDOISE
FRANCE_PIERRE_GRES_ARDOISE FRANCE_PIERRE_GRES_ARDOISE
FRANCE_PIERRE_GRES_ARDOISE FRANCE_PIERRE_GRES_ARDOISE
FRANCE_PIERRE_GRES_ARDOISE FRANCE_PIERRE_GRES_ARDOISE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PIERRE_GRES_TUILE
FRANCE_PIERRE_GRES_TUILE FRANCE_PI

FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_ZINC FRANCE_PIERRE_CALCAIRE_ZINC
FRANCE_PIERRE_GNEISS_TUILE FRANCE_PIERRE_GNEISS_TUILE
FRANCE_PIERRE_GNEISS

FRANCE_BRIQUE_ARDOISE FRANCE_BRIQUE_ARDOISE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_BRIQUE_TUILE FRANCE_BRIQUE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_

FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_TUILE FRANCE_PIERRE_CALCAIRE_TUILE
FRANCE_PIERRE_CALCAIRE_ZINC FRANCE_PIERRE_CALCAIRE_ZINC
FRANCE_PIERRE_GNEISS_TUILE FRANCE_PIERRE_GNEISS_TUILE
FRANCE_PIERRE_GNEISS_TUILE FRANCE_PIERRE_GNEISS_TUILE
FRANCE_PIERRE_GNEISS_TUI

...

FRANCE_ARDOISE FRANCE_ARDOISE
FRANCE_ARDOISE FRANCE_ARDOISE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRAN

FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE

FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE

FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE

FRANCE_ARDOISE FRANCE_ARDOISE
FRANCE_ARDOISE FRANCE_ARDOISE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRANCE_TUILE FRANCE_TUILE
FRAN

In [69]:
problem_ter

[]

### Dept problems

#### Dept different in file name and insee

In [60]:
# Checking dept
folder_new_csvs = r"C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept"
list_csvs = os.listdir(folder_new_csvs)
problem_dept = []

for csv_name in list_csvs:
    file = pd.read_csv(fr"{folder_new_csvs}\{csv_name}",nrows=1)
    (file.columns)
    dept_nb = str(file.INSEE_DEP_min.unique()[0])
    if len(dept_nb) == 1:
        dept_nb = "0" + dept_nb
    dept_name = csv_name.split("-")[4][3:]
    if len(dept_name) == 1:
        dept_name = "0" + dept_name
    print(dept_nb, dept_name)
    
    if dept_nb != dept_name:
        
        print(csv_name)
        display(file[["INSEE_DEP_min"]].head())
        problem_dept.append(csv_name)
        

31 31
59 59
34 34
42 42
64 64
81 81
74 74
59 59
60 60
69 69
59 59
78 78
19 19
06 06
69 69
40 40
73 73
74 74
53 53
76 76
21 21
31 31
45 45
54 54
59 59
61 61
62 62
65 65
71 71
72 72
78 78
79 79
80 80
85 85
91 91
92 92
93 93
94 94
95 95
19 19
37 37
04 04
06 06
11 11
13 13
16 16
17 17
02 02
25 25
27 27
28 28
30 30
33 33
34 34
36 36
38 38
41 41
42 42
46 46
55 55
60 60
69 69
70 70
77 77
83 83
86 86
75 75
47 47
82 82
84 84
29 29
49 49
56 56
14 14
23 23
2B 2B
87 87
22 22
35 35
44 44
50 50
07 07
18 18
03 03
43 43
57 57
64 64
67 67
81 81
88 88
05 05
10 10
24 24
26 26
68 68
08 08
32 32
66 66
63 63
01 01
39 39
51 51
52 52
40 40
73 73
74 74
53 53
76 76
21 21
31 31
45 45
54 54
59 59
61 61
62 62
65 65
71 71
72 72
78 78
79 79
80 80
85 85
89 89
90 90
91 91
92 92
93 93
94 94
95 95
19 19
37 37
48 48
04 04
06 06
11 11
13 13
16 16
17 17
02 02
25 25
27 27
28 28
30 30
33 33
34 34
36 36
38 38
41 41
42 42
46 46
55 55
60 60
69 69
70 70
77 77
83 83
86 86
75 75
47 47
82 82
84 84
12 12
29 29
49 49
56 56
14 14
23 2

74 74
77 77
78 78
79 79
08 08
80 80
81 81
83 83
84 84
86 86
87 87
90 90
91 91
92 92
93 93
94 94
95 95
75 75
44 44
56 56
06 06
13 13
14 14
16 16
18 18
21 21
26 26
28 28
30 30
31 31
33 33
34 34
36 36
38 38
51 51
54 54
57 57
59 59
60 60
62 62
63 63
64 64
67 67
69 69
72 72
74 74
78 78
79 79
81 81
83 83
84 84
87 87
90 90
91 91
92 92
93 93
94 94
95 95
12 12
19 19
22 22
29 29
35 35
37 37
44 44
49 49
50 50
53 53
56 56
76 76
01 01
04 04
05 05
06 06
07 07
09 09
10 10
11 11
13 13
14 14
15 15
16 16
17 17
18 18
02 02
21 21
24 24
26 26
27 27
28 28
03 03
30 30
31 31
33 33
34 34
36 36
38 38
39 39
40 40
41 41
42 42
43 43
45 45
46 46
47 47
51 51
52 52
54 54
55 55
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
77 77
78 78
79 79
08 08
80 80
81 81
83 83
84 84
85 85
86 86
87 87
88 88
89 89
91 91
92 92
93 93
94 94
95 95
75 75
12 12
19 19
22 22
29 29
35 35
37 37
44 44
48 48
49 49
50 50
56 56
76 76
01 01
04 04
05 05
06 06
07 07
09 09
10 10
11 11
13 13
14 1

63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
77 77
78 78
79 79
08 08
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
75 75
12 12
19 19
22 22
29 29
35 35
37 37
44 44
48 48
49 49
50 50
53 53
56 56
76 76
01 01
04 04
05 05
06 06
07 07
09 09
10 10
11 11
13 13
14 14
15 15
16 16
17 17
18 18
02 02
21 21
23 23
24 24
25 25
26 26
27 27
28 28
2A 2A
2B 2B
03 03
30 30
31 31
32 32
33 33
34 34
36 36
38 38
39 39
40 40
41 41
42 42
43 43
45 45
46 46
47 47
51 51
52 52
54 54
55 55
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
77 77
78 78
79 79
08 08
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
75 75
37 37
44 44
49 49
56 56
76 76
06 06
14 14
25 25
30 30
31 31
33 33
34 34
38 38
42 42
45 45
57 57
59 59
60 60
62 62
63 63
64 64
65 65
69 69
71 71
72 72
73 73
08 08
83 83
84 84
91 91
92 92
93 93
94 94
95 95
75 75
91 91
37 37
44 4

84 84
86 86
87 87
90 90
91 91
92 92
93 93
94 94
95 95
75 75
22 22
37 37
76 76
06 06
10 10
14 14
16 16
02 02
21 21
26 26
28 28
31 31
33 33
34 34
38 38
42 42
45 45
51 51
54 54
57 57
59 59
62 62
63 63
64 64
65 65
67 67
68 68
69 69
71 71
77 77
78 78
81 81
83 83
84 84
86 86
87 87
91 91
92 92
93 93
94 94
95 95
75 75
12 12
19 19
22 22
29 29
35 35
37 37
44 44
49 49
50 50
53 53
56 56
76 76
01 01
04 04
05 05
06 06
07 07
10 10
13 13
14 14
17 17
18 18
02 02
21 21
25 25
26 26
27 27
28 28
2B 2B
03 03
30 30
31 31
33 33
34 34
36 36
38 38
40 40
41 41
42 42
43 43
45 45
46 46
47 47
51 51
52 52
57 57
58 58
59 59
60 60
62 62
63 63
64 64
65 65
66 66
68 68
69 69
71 71
72 72
73 73
74 74
77 77
78 78
79 79
80 80
82 82
83 83
84 84
85 85
86 86
87 87
89 89
90 90
91 91
92 92
93 93
94 94
95 95
75 75
12 12
19 19
22 22
29 29
35 35
37 37
44 44
48 48
49 49
50 50
53 53
56 56
76 76
01 01
04 04
05 05
06 06
07 07
09 09
10 10
11 11
13 13
14 14
15 15
16 16
17 17
18 18
02 02
21 21
23 23
24 24
25 25
26 26
27 27
28 28
30 30
31 3

02 02
21 21
23 23
24 24
25 25
26 26
27 27
28 28
2A 2A
2B 2B
03 03
30 30
31 31
32 32
33 33
34 34
36 36
38 38
39 39
40 40
41 41
42 42
43 43
45 45
46 46
47 47
51 51
52 52
54 54
55 55
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
77 77
78 78
79 79
08 08
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
75 75
12 12
19 19
22 22
29 29
35 35
37 37
44 44
48 48
49 49
50 50
53 53
56 56
76 76
01 01
04 04
05 05
06 06
07 07
09 09
10 10
11 11
13 13
14 14
15 15
16 16
17 17
18 18
02 02
21 21
23 23
24 24
25 25
26 26
27 27
28 28
2A 2A
2B 2B
03 03
30 30
31 31
32 32
33 33
34 34
36 36
38 38
39 39
40 40
41 41
42 42
43 43
45 45
46 46
47 47
51 51
52 52
54 54
55 55
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
77 77
78 78
79 79
08 08
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
75 75
50 50
44 4

In [61]:
len(problem_dept)

0

In [62]:
problem_dept

[]

In [ ]:
for csv_prob in problem_dept:
    source = fr"{folder_new_csvs}\{csv_prob}"
    destination = fr"{folder_new_csvs}\problem\{csv_prob}"
    shutil.copy(source, destination)

In [47]:
for csv_to_change in os.listdir(fr"{folder_new_csvs}\problem"):
    file = pd.read_csv(fr"{folder_new_csvs}\{csv_to_change}")
    (file.columns)
    dept_nb = str(file.INSEE_DEP_min.unique()[0])
    if len(dept_nb) == 1:
        dept_nb = "0" + dept_nb
    dept_name = csv_name.split("-")[4][3:]
    if len(dept_name) == 1:
        dept_name = "0" + dept_name
    print(dept_nb, dept_name)
    
    

    
    if dept_nb != dept_name:
        
        print("\nold name: ", csv_to_change)
        arch = file.archetype.unique()[0]
        name_corrected = fr"{folder_new_csvs}\solved\{arch}-dep{dept_nb}-len{len(file)}.csv"
        print("new name",  name_corrected)
        display(file[["INSEE_DEP_min", 'typology',
             'usage',
             'territory',
             'period','Ter_P1_min',
             'Ter_P2-7_min',]].head())
        file.to_csv(name_corrected)
        print("*"*50)
        
        

27 95

old name:  P1-BA-HABITAT-FRANCE_PIERRE_CALCAIRE_ARDOISE-dep76-len1.csv
new name C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept\solved\P1-BA-HABITAT-FRANCE_PIERRE_CALCAIRE_ARDOISE-dep27-len1.csv


,INSEE_DEP_min,typology,usage,territory,period,Ter_P1_min,Ter_P2-7_min
0,27,BA,HABITAT,FRANCE_PIERRE_CALCAIRE_ARDOISE,P1,FRANCE_PIERRE_CALCAIRE_ARDOISE,FRANCE_ARDOISE


**************************************************
69 95

old name:  P1-P-COMMERCE-FRANCE_PIERRE_CALCAIRE_TUILE-dep01-len1.csv
new name C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept\solved\P1-P-COMMERCE-FRANCE_PIERRE_CALCAIRE_TUILE-dep69-len1.csv


,INSEE_DEP_min,typology,usage,territory,period,Ter_P1_min,Ter_P2-7_min
0,69,P,COMMERCE,FRANCE_PIERRE_CALCAIRE_TUILE,P1,FRANCE_PIERRE_CALCAIRE_TUILE,FRANCE_TUILE


**************************************************
73 95

old name:  P1-P-HABITAT-FRANCE_BOIS_ARDOISE-dep74-len3.csv
new name C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept\solved\P1-P-HABITAT-FRANCE_BOIS_ARDOISE-dep73-len3.csv


...

**************************************************
78 95

old name:  P7-P-HABITAT-FRANCE_ARDOISE-dep92-len1.csv
new name C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept\solved\P7-P-HABITAT-FRANCE_ARDOISE-dep78-len1.csv


,INSEE_DEP_min,typology,usage,territory,period,Ter_P1_min,Ter_P2-7_min
0,78,P,HABITAT,FRANCE_ARDOISE,P7,FRANCE_BRIQUE_ARDOISE,FRANCE_ARDOISE


**************************************************
78 95

old name:  P7-P-HABITAT-FRANCE_ARDOISE-dep95-len2.csv
new name C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept\solved\P7-P-HABITAT-FRANCE_ARDOISE-dep78-len2.csv


,INSEE_DEP_min,typology,usage,territory,period,Ter_P1_min,Ter_P2-7_min
0,78,P,HABITAT,FRANCE_ARDOISE,P7,FRANCE_BRIQUE_ARDOISE,FRANCE_ARDOISE
1,78,P,HABITAT,FRANCE_ARDOISE,P7,FRANCE_BRIQUE_ARDOISE,FRANCE_ARDOISE


**************************************************


#### Multiple insee depts in a same file

In [41]:
# Checking dept
folder_new_csvs = r"C:\Users\ldecarva\Documents\Data\gpd_sorting_output\new_territory_archs_per_dept"
list_csvs = os.listdir(folder_new_csvs)
problem_dept_non_unique = []
i = 0 
for csv_name in list_csvs:
    file = pd.read_csv(fr"{folder_new_csvs}\{csv_name}")
    insee_dep = file.INSEE_DEP_min.unique()
    if len(insee_dep) > 1:
        i+=1
        print("number", i)
        print(csv_name)
        print(len(insee_dep), "depts:   ",insee_dep)
#         display(file[["INSEE_DEP_min"]].head(2))
#         problem_dept_non_unique.append([insee_dep,csv_name])
    
        dict_dep_lens = {}
        for el in insee_dep:
            total_len = len(file)
            len_dep = len(file[file["INSEE_DEP_min"] == el])
            dict_dep_lens[el] = [f"{len_dep / total_len * 100 :.3f}%", len_dep]
        problem_dept_non_unique.append([csv_name, dict_dep_lens])
        
        print("*"*50, "\n")


number 1
P1-BA-HABITAT-FRANCE_BRIQUE_TUILE-dep62-len367.csv
2 depts:    [62 59]
************************************************** 

number 2
P1-I-COMMERCE-FRANCE_BRIQUE_TUILE-dep62-len385.csv
2 depts:    [62 59]
************************************************** 

number 3
P1-I-COMMERCE-FRANCE_BRIQUE_TUILE-dep95-len258.csv
2 depts:    [95 93]
************************************************** 

number 4
P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep65-len5354.csv
2 depts:    [65 32]
************************************************** 

number 5
P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep90-len2378.csv
2 depts:    [90 25]
************************************************** 

number 6
P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep92-len12356.csv
2 depts:    [92 75]
************************************************** 

number 7
P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep93-len9366.csv
4 depts:    [93 75 77 92]
************************************************** 

number 8
P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep94-len13713.csv
2

number 62
P1-P-HABITAT-FRANCE_PIERRE_CALCAIRE_TUILE-dep60-len92592.csv
2 depts:    [60  2]
************************************************** 

number 63
P1-P-HABITAT-FRANCE_PIERRE_CALCAIRE_TUILE-dep61-len6.csv
3 depts:    [61 27 76]
************************************************** 

number 64
P1-P-HABITAT-FRANCE_PIERRE_CALCAIRE_TUILE-dep69-len71940.csv
4 depts:    [69 42 38  1]
************************************************** 

number 65
P1-P-HABITAT-FRANCE_PIERRE_CALCAIRE_TUILE-dep70-len42661.csv
2 depts:    [70 25]
************************************************** 

number 66
P1-P-HABITAT-FRANCE_PIERRE_CALCAIRE_TUILE-dep83-len51597.csv
2 depts:    [83  4]
************************************************** 

number 67
P1-P-HABITAT-FRANCE_PIERRE_GNEISS_TUILE-dep47-len5.csv
3 depts:    [47 32 82]
************************************************** 

number 68
P1-P-HABITAT-FRANCE_PIERRE_GNEISS_TUILE-dep47-len56165.csv
2 depts:    [47 24]
*********************************************

number 123
P2-P-HABITAT-FRANCE_TUILE-dep71-len23043.csv
3 depts:    [71 21 69]
************************************************** 

number 124
P2-P-HABITAT-FRANCE_TUILE-dep72-len29596.csv
2 depts:    [72 49]
************************************************** 

number 125
P2-P-HABITAT-FRANCE_TUILE-dep80-len33897.csv
3 depts:    [80 76 62]
************************************************** 

number 126
P2-P-HABITAT-FRANCE_TUILE-dep83-len51355.csv
2 depts:    [83  6]
************************************************** 

number 127
P2-P-HABITAT-FRANCE_TUILE-dep84-len26513.csv
2 depts:    [84 26]
************************************************** 

number 128
P2-P-HABITAT-FRANCE_TUILE-dep85-len50804.csv
2 depts:    [85 79]
************************************************** 

number 129
P2-P-HABITAT-FRANCE_TUILE-dep87-len20314.csv
3 depts:    [87 24 19]
************************************************** 

number 130
P2-P-HABITAT-FRANCE_TUILE-dep89-len19730.csv
2 depts:    [89 77]
************

number 187
P5-P-HABITAT-FRANCE_TUILE-dep24-len14545.csv
2 depts:    [24 46]
************************************************** 

number 188
P5-P-HABITAT-FRANCE_TUILE-dep34-len47702.csv
2 depts:    [34 30]
************************************************** 

number 189
P5-P-HABITAT-FRANCE_TUILE-dep39-len6343.csv
2 depts:    [39 71]
************************************************** 

number 190
P5-P-HABITAT-FRANCE_TUILE-dep41-len9182.csv
2 depts:    [41 28]
************************************************** 

number 191
P5-P-HABITAT-FRANCE_TUILE-dep46-len7754.csv
2 depts:    [46 24]
************************************************** 

number 192
P5-P-HABITAT-FRANCE_TUILE-dep52-len3142.csv
2 depts:    [52 10]
************************************************** 

number 193
P5-P-HABITAT-FRANCE_TUILE-dep54-len11466.csv
2 depts:    [54 57]
************************************************** 

number 194
P5-P-HABITAT-FRANCE_TUILE-dep62-len27430.csv
2 depts:    [62 59]
*************************

number 252
P7-P-HABITAT-FRANCE_TUILE-dep93-len2589.csv
2 depts:    [93 77]
************************************************** 

number 253
P7-P-HABITAT-FRANCE_TUILE-dep95-len6389.csv
2 depts:    [95 93]
************************************************** 



In [42]:
(len(problem_dept_non_unique))

253

In [58]:
level    = logging.INFO
format   = '  %(message)s'
handlers = [logging.FileHandler('problem_dept_non_unique.log'), logging.StreamHandler()]
logging.basicConfig(level = level, format = format, handlers = handlers)

logging.info("Files with non unique departments")
logging.info(f"Total number of files with problems: {len(problem_dept_non_unique)}")
logging.info(f"Percentage of files : {len(problem_dept_non_unique)/len(list_csvs) *100 :.1f} %\n")

for elem in problem_dept_non_unique:
    logging.info(elem[0])
    for k, v in elem[1].items():
        logging.info(k)
        logging.info(v)

    logging.info("\n")
logging.shutdown()

  Files with non unique departments
  Total number of files with problems: 253
  Percentage of files : 3.6 %

  P1-BA-HABITAT-FRANCE_BRIQUE_TUILE-dep62-len367.csv
  62
  ['99.728%', 366]
  59
  ['0.272%', 1]
  

  P1-I-COMMERCE-FRANCE_BRIQUE_TUILE-dep62-len385.csv
  62
  ['99.740%', 384]
  59
  ['0.260%', 1]
  

  P1-I-COMMERCE-FRANCE_BRIQUE_TUILE-dep95-len258.csv
  95
  ['99.612%', 257]
  93
  ['0.388%', 1]
  

  P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep65-len5354.csv
  65
  ['99.981%', 5353]
  32
  ['0.019%', 1]
  

  P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep90-len2378.csv
  90
  ['99.958%', 2377]
  25
  ['0.042%', 1]
  

  P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep92-len12356.csv
  92
  ['99.927%', 12347]
  75
  ['0.073%', 9]
  

  P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep93-len9366.csv
  93
  ['99.925%', 9359]
  75
  ['0.043%', 4]
  77
  ['0.011%', 1]
  92
  ['0.021%', 2]
  

  P1-I-HABITAT-FRANCE_BRIQUE_TUILE-dep94-len13713.csv
  94
  ['99.978%', 13710]
  75
  ['0.022%', 3]
  

  P1-I-HABITAT-FRANCE_B

  82
  ['99.992%', 36438]
  12
  ['0.008%', 3]
  

  P1-P-HABITAT-FRANCE_PIERRE_GNEISS_TUILE-dep84-len45601.csv
  84
  ['99.993%', 45598]
  26
  ['0.007%', 3]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_ARDOISE-dep49-len93091.csv
  49
  ['99.998%', 93089]
  44
  ['0.002%', 2]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_ARDOISE-dep56-len79960.csv
  56
  ['99.997%', 79958]
  22
  ['0.001%', 1]
  29
  ['0.001%', 1]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_ARDOISE-dep87-len2.csv
  87
  ['50.000%', 1]
  24
  ['50.000%', 1]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_TUILE-dep14-len5.csv
  14
  ['60.000%', 3]
  61
  ['40.000%', 2]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_TUILE-dep23-len47410.csv
  23
  ['99.998%', 47409]
  3
  ['0.002%', 1]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_TUILE-dep36-len2.csv
  36
  ['50.000%', 1]
  23
  ['50.000%', 1]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_TUILE-dep86-len3.csv
  86
  ['33.333%', 1]
  19
  ['66.667%', 2]
  

  P1-P-HABITAT-FRANCE_PIERRE_GRANITE_TUILE-dep87

  ['99.980%', 4914]
  87
  ['0.020%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep27-len26439.csv
  27
  ['99.996%', 26438]
  14
  ['0.004%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep31-len34515.csv
  31
  ['99.997%', 34514]
  9
  ['0.003%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep38-len36343.csv
  38
  ['99.997%', 36342]
  26
  ['0.003%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep39-len10757.csv
  39
  ['99.981%', 10755]
  25
  ['0.009%', 1]
  1
  ['0.009%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep45-len33388.csv
  45
  ['99.997%', 33387]
  18
  ['0.003%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep52-len7364.csv
  52
  ['99.986%', 7363]
  21
  ['0.014%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep61-len12169.csv
  61
  ['99.992%', 12168]
  14
  ['0.008%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep62-len50113.csv
  62
  ['99.998%', 50112]
  59
  ['0.002%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep64-len20517.csv
  64
  ['99.995%', 20516]
  40
  ['0.005%', 1]
  

  P3-P-HABITAT-FRANCE_TUILE-dep69-len27802.csv
  69
  ['

  25
  ['0.024%', 1]
  

  P6-P-HABITAT-FRANCE_TUILE-dep93-len6515.csv
  93
  ['99.877%', 6507]
  77
  ['0.123%', 8]
  

  P6-P-HABITAT-FRANCE_TUILE-dep94-len5819.csv
  94
  ['99.966%', 5817]
  93
  ['0.017%', 1]
  92
  ['0.017%', 1]
  

  P7-I-HABITAT-FRANCE_TUILE-dep92-len957.csv
  92
  ['99.896%', 956]
  75
  ['0.104%', 1]
  

  P7-I-HABITAT-FRANCE_TUILE-dep94-len1003.csv
  94
  ['99.900%', 1002]
  75
  ['0.100%', 1]
  

  P7-I-HABITAT-FRANCE_TUILE-dep95-len1152.csv
  95
  ['99.913%', 1151]
  93
  ['0.087%', 1]
  

  P7-P-HABITAT-FRANCE_TUILE-dep09-len3111.csv
  9
  ['99.968%', 3110]
  11
  ['0.032%', 1]
  

  P7-P-HABITAT-FRANCE_TUILE-dep17-len20622.csv
  17
  ['99.995%', 20621]
  33
  ['0.005%', 1]
  

  P7-P-HABITAT-FRANCE_TUILE-dep31-len24799.csv
  31
  ['99.996%', 24798]
  9
  ['0.004%', 1]
  

  P7-P-HABITAT-FRANCE_TUILE-dep32-len4233.csv
  32
  ['99.953%', 4231]
  31
  ['0.047%', 2]
  

  P7-P-HABITAT-FRANCE_TUILE-dep34-len22288.csv
  34
  ['99.996%', 22287]
  11
  ['0.004%',

##### Testing one case

In [76]:
file_to_study = problem_dept_non_unique[218][0]
file_to_study

'P6-P-HABITAT-FRANCE_TUILE-dep61-len10796.csv'

In [66]:
path_file_to_study = Path(folder_new_csvs) / file_to_study
file = pd.read_csv(path_file_to_study)
file.head(1)

,Unnamed: 0,territory,Unnamed: 0.1,typology,usage,period,bnb_id,altitude_sol,adr_fiabilite_niv_1,adr_fiabilite_niv_2,...,Ter_P1_min,Ter_P2-7_min,TYPO_M,geometry,TYPO_M_S,TYPO_BDNB_S,archetype,L1,L2,L3
0,0,FRANCE_TUILE,7,P,HABITAT,P6,612870000A0154_e6d25e8f3e5cce9,101.0,problème de géocodage,batiment non géocodée au numéro,...,FRANCE_BRIQUE_TUILE,FRANCE_TUILE,NaN,"MULTIPOLYGON (((437409.3 6864912.3, 437414.1 6...",NaN,P,P6-P-HABITAT-FRANCE_TUILE,1,1,1


In [67]:
list(file.columns)

['Unnamed: 0',
 'territory',
 'Unnamed: 0.1',
 'typology',
 'usage',
 'period',
 'bnb_id',
 'altitude_sol',
 'adr_fiabilite_niv_1',
 'adr_fiabilite_niv_2',
 'config_adr',
 'geombui_area',
 'cerffo2020_l_adresse',
 'adedpe202006_logtype_min_classe_ener_ges',
 'adedpe202006_logtype_coherence_data_methode_dpe',
 'adedpe202006_logtype_is_3cl',
 'adedpe202006_logtype_ratio_ges_conso',
 'adedpe202006_logtype_ch_gen_lib',
 'adedpe202006_logtype_ecs_gen_lib',
 'adedpe202006_min_conso_ener',
 'adedpe202006_min_estim_ges',
 'adedpe202006_max_conso_ener',
 'adedpe202006_max_estim_ges',
 'adedpe202006_l_ch_gen_princ',
 'adedpe202006_l_ecs_gen_princ',
 'anarnc202012_nb_log',
 'anarnc202012_nb_lot_garpark',
 'anarnc202012_nb_lot_tertiaire',
 'anarnc202012_nb_lot_tot',
 'anarnc202012_l_nom_copro',
 'cerdvf2021v5_min_valeur_fonc_par_m2_hab',
 'cerdvf2021v5_max_valeur_fonc_par_m2_hab',
 'cerdvf2021v5_med_valeur_fonc_par_m2_hab',
 'cerdvf2021v5_mean_valeur_fonc_par_m2_hab_t1',
 'cerdvf2021v5_mean_valeur

In [70]:
file[['code_insee',
 'code_iris',
 'commune',
 'departement',
 'etaban202111_label',"INSEE_DEP_min"]].head()

,code_insee,code_iris,commune,departement,etaban202111_label,INSEE_DEP_min
0,61287,612870000.0,MONTILLY-SUR-NOIREAU,61,Residence les Champs Saint Denis 61100 Montill...,61
1,61309,613090000.0,PERCHE EN NOCE,61,Lieu Dit Sinevaux 61340 Perche en Nocé,61
2,61309,613090000.0,PERCHE EN NOCE,61,Lieu Dit la Joannerie 61340 Perche en Nocé,61
3,61074,610740000.0,CARROUGES,61,Les Vergers 61320 Carrouges,61
4,61074,610740000.0,CARROUGES,61,Les Vergers 61320 Carrouges,61


In [71]:
file.departement.unique()

array([61], dtype=int64)

In [72]:
file.INSEE_DEP_min.unique()

array([61, 53, 27, 14], dtype=int64)

In [87]:
file.groupby(["departement","INSEE_DEP_min"]).size().sort_values(ascending=False)
# For this file, the variable from Mapuce (INSEE_DEP_min) have multiple depts. 
# The variable from CSTB (departement )does not have a problem.
# Check the other variables

departement  INSEE_DEP_min
61           61               10792
             53                   2
             14                   1
             27                   1
dtype: int64

In [77]:
len(file)

10796

#### Checking for multiple CSTB departments in a same file

In [93]:
start_time_tot = time()
# Checking dept
folder_new_csvs = r"C:\Users\ldecarva\Documents\Data\gpd_sorting_output\csvs_per_department"
list_csvs = os.listdir(folder_new_csvs)

problem_cstb_dpt_non_unique = []
i = 0 
for csv_name in list_csvs:
    start_time_dep = time()

    file = pd.read_csv(fr"{folder_new_csvs}\{csv_name}")
    cstb_dep = file.departement.unique()
    if len(cstb_dep) > 1:
        i+=1
        print("number", i)
        print(csv_name)
        print(len(cstb_dep), "cstb_depts:   ",cstb_dep)

    
        dict_dep_lens = {}
        for el in cstb_dep:
            total_len = len(file)
            len_dep = len(file[file["departement"] == el])
            dict_dep_lens[el] = [f"{len_dep / total_len * 100 :.3f}%", len_dep]
        problem_cstb_dpt_non_unique.append([csv_name, dict_dep_lens])
        
        print("*"*50, "\n")
    else:
        print(f"{cstb_dep} => OK!")
    dur_dep = time() - start_time_dep
    print(f"Time to open: {dur_dep:.1f} \n")

dur_tot = time() - start_time_tot
print(f"Time TOTAL: {dur_tot:.1f} \n")

[1] => OK!
Time to open: 5.9 



C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (24,120,142,186,286,287,288,290,295,317,323) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[2] => OK!
Time to open: 5.8 



C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (24,120,137,142,186,286,287,288,295) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[3] => OK!
Time to open: 4.4 



...

[93] => OK!
Time to open: 4.9 



C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (142,186,286,287,288,295) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[94] => OK!
Time to open: 4.4 



C:\Users\ldecarva\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (62,120,142,186,235,286,287,288,295,317,323) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


[95] => OK!
Time to open: 6.3 

Time TOTAL: 580.6 



In [95]:
(len(problem_cstb_dpt_non_unique))

0

In [97]:
580/60

9.666666666666666